# Project 2

Morgan Elder<br>
Ben Vuong<br>
CS 4320

1. Write a brief (no more than a paragraph) note on the importance of the choice of 
parameter settings in evolutionary algorithms. Or argue, with well-articulated, precise 
reasons (one paragraph only), that parameters do not really matter in EC.  

I would say that when working with evolutionary algorithms, diversity is very important for the growth and success of the evolutionary algorithms. There should not be one indiviudual within the population that over takes the other. If that would happen then there would be stagnation within the growth of the algorithms. In order to prevent such an event of happening and encourage growth and diversity, it is important to focus on the choice of parameters settings in evolutionary algorithms. If good parameters were choosen, there will be higher chances or growth and diversity to happen and the inverse would result in a higher chance of stangnation.

Evolutionary algorithm (EA) parameters directly influence the exploration with diverse populations and exploitation of fit individuals. Parameters include all of the algorithm design choices such as genetic operators, operator parameters, stopping conditions, population size and seed. For example, parameters like the mutation rate or cross over probability can increase exploration whereas selection/sampling method can increase exploitation. Because EAs are stochastic, the applicability of an EA, more specfically the EA parameters, accross different problems, problem instances, and sizes must be compared using statistical methods and many runs that are *iid*. Therefore, parameter settings are an important part of the optimization process that must strike a balance between generalizability, computational cost, and performance.

# De Jong Function 5: Variation of Shekel's Function 

The objective of this project is to minimize Shekel's function using parameters from De Jong function #5:

$$f(\vec{x}) = \left(0.002 + \sum_{i=1}^{25}\frac{1}{i + (x_1 - a_{1i})^6 + (x_2 - a_{2i})^6}\right)^{-1}$$

where $$
\textbf{a} =
 \begin{pmatrix}
    -32 & -16 & 0 & 16 & 32 & -32 & \ldots & 0 & 16 & 32 \\
    -32 & -32 & -32 & -32 & -32 & -16 & \ldots & 32 & 32 & 32
 \end{pmatrix}$$

The number of dimensions is 2 and the input domain is $-65.536 \le x_i \le 65.536$ for $i=1,2$. 










In [147]:
#import modules
import numpy as np
from enum import Enum
import warnings
import inspect
from typing import Union

## Optimization Goals

Use the Optimization_Goal class as an enum type in order to define a goal variable as either min or max.

In [148]:
class Optimization_Goal(Enum):
  MINIMIZE = 1
  MAXIMIZE = 2

## Optimization Problems

Genetic algorithms are suited for optimization problems. The Problem class is used to identify subclasses as optimization problems.

In [149]:
class Problem():
  """The Problem class defines the problems contains the definitions of
  of problems. Problems include any information such as the optimization
  goal (min/max), objective function, input dimension, domain/range constraints, 
  and more."""
  pass
  
class De_Jong_Function_5(Problem):
  """De Jong function #5 is variation of Shekel's foxeholes problem involving
  25 local minima, 2 dimensions, and predefined constants.
  This problem is suited for multimodal optimization."""
  dimensions=2
  goal=Optimization_Goal.MINIMIZE
  upper_bounds = [65.536, 65.536]
  lower_bounds = [-65.536, 65.536]
  constants_array = np.array(
      [[-32., -16., 0., 16., 32., -32., -16., 0., 16., 32., -32., -16., 0., 
        16., 32., -32., -16., 0., 16., 32., -32., -16., 0., 16., 32.],
        [-32., -32., -32., -32., -32., -16., -16., -16., -16., -16., 0., 0., 
        0., 0., 0., 16., 16., 16., 16., 16., 32., 32., 32., 32., 32.]]
    )
  def objective_function(self, X):
    # array representing the 1st to 25th elements of the summation
    array_i = np.arange(1, self.constants_array.shape[1]+1, step=1)
    results = (0.002 + np.sum(1/(array_i 
                        + (X[:,[0]] - self.constants_array[0])**6
                        + (X[:, [1]] - self.constants_array[1])**6), axis=1))**-1
    return results    

## Genetic Operators

In [150]:
class Genetic_Operator:
    """The Genetic_Operator base class defines the genetic operators used in
    genetic algorithms. These include crossover and mutation."""
    pass

### Selection

In [151]:
class Selection(Genetic_Operator):
    """The Selection class defines the selection operator used in genetic
    algorithms. This includes the selection of parents and the selection of
    survivors."""
    
    def __init__(self, selection_type: str="proportional"):
      # dictionary of selection types and their corresponding methods
      selection_types = { "proportional": self.proportional }
      # check if selection type is valid
      if selection_type not in selection_types:
        raise ValueError(f"Invalid selection type: {selection_type}")
      self.selection_type = selection_types[selection_type]
    

    
    def proportional(self):
      print("hello from proportional")
      pass

    def run(self, **kwargs):
      """The run method runs the selection operator."""
      self.selection_type(**kwargs)
        

### Cross Over

In [152]:
class Crossover(Genetic_Operator):
    """The Crossover class defines the crossover operator used in genetic
    algorithms."""
    
    def __init__(self):
      pass

### Mutation

In [153]:
class Mutation(Genetic_Operator):
    """The Mutation class defines the mutation operator used in genetic
    algorithms."""
    
    def __init__(self):
      pass

## Stopping Conditions

In [154]:
class Stopping_Condition:
    """The Stopping_Condition base class defines the stopping conditions used
    in genetic algorithms. These include the number of generations and the
    fitness threshold."""

    
    pass

class Max_Generations(Stopping_Condition):
    """The number_of_generations class defines the stopping condition based on
    the number of generations."""
    def __init__(self, max_generations: int=100):
      self.max_generations = max_generations
    
    def is_running(self, generation: int):
        """The is_running method checks if the algorithm should continue running
        based on the number of generations."""
        running = True
        if generation >= self.max_generations:
            running = False
        return running  
    
    def increase_max_generations(self, increase: int=100):
        """The increase_max_generations method increases the maximum number of
        generations by a specified amount."""
        self.max_generations += increase

    pass

## Genetic Algorithm

In [155]:
class Genetic_Algorithm():
    problem : Problem
    algorithm_steps : list[Genetic_Operator] = []
    stopping_condition : Stopping_Condition
    generation : int = 0
    
    def __init__(self, problem : Problem = None, algorithm_steps : list[Genetic_Operator] = [],
                 stopping_condition : Stopping_Condition = Max_Generations(),
                 population_size : int = 100, population : np.ndarray = None,):
        # loop locals and validate types
        for key, value in locals().items():
            match key:
                case "problem":
                    # check if problem is an instance of Problem
                    if (not isinstance(value, Problem)):
                        raise TypeError(f"problem must be an instance of Problem. Received {type(value)}")
                    self.problem = value
                case "algorithm_steps":
                    # check if algorithm_steps is a list
                    if (not isinstance(value, list)):
                        raise TypeError(f"algorithm_steps must be a list. Received {type(value)}")
                    # check if algorithm_steps is a list of Genetic_Operator
                    if (not all(isinstance(item, Genetic_Operator) for item in value)):
                        raise TypeError(f"algorithm_steps must be a list of Genetic_Operator. Received {type(value)}")
                    self.algorithm_steps = value
                case "stopping_condition":
                    # check if stopping_condition is an instance of Stopping_Condition
                    if (not isinstance(value, Stopping_Condition)):
                        raise TypeError(f"stopping_condition must be an instance of Stopping_Condition. Received {type(value)}")
                    self.stopping_condition = value
            

    def add_step(self, step : Genetic_Operator):
        """Add a step to the algorithm. The step will be appended to the
        end of the algorithm."""
        # check if step is a Genetic_Operator
        if (not isinstance(step, Genetic_Operator)):
            raise TypeError(f"step must be a Genetic_Operator. Received {type(step)}")
        self.algorithm_steps.append(step)

    def add_steps(self, steps : list[Genetic_Operator]):
        """Add multiple steps to the algorithm. The steps will be appended to the
        end of the algorithm."""
        # check if steps is a list
        if (not isinstance(steps, list)):
            raise TypeError(f"steps must be a list. Received {type(steps)}")
        # check if steps is a list of Genetic_Operator
        if (not all(isinstance(item, Genetic_Operator) for item in steps)):
            raise TypeError(f"steps must be a list of Genetic_Operator. Received {type(steps)}")
        self.algorithm_steps.extend(steps)

    def set_problem(self, problem : Problem):
        """Set the problem to be solved. If a problem has already been set,
        it will be overwritten and a warning will be raised."""
        if (self.problem is not None):
            # 
            warnings.warn(f"Problem already set to {self.problem}. Overwriting with {problem}.")
        self.problem = problem

    def run(self):
        """Run the genetic algorithm."""
        if (self.is_ready()):
            # self.evaluate()
            # run algorithm steps
            while (self.is_running()):
                for step in self.algorithm_steps:
                    # build arg dict for algorithm step and then run it
                    step_args = self.build_arg_dict(step.run)
                    step.run(**step_args)
                    # self.evaluate()
                    # record metrics
                    pass
                self.generation += 1
    
    def build_arg_dict(self, func):
        """Build an argument dictionary for a function."""
        signature = inspect.signature(func)
        # build arg dict for algorithm step
        args = {}
        for param in signature.parameters.values():
            if param.name == "self":
                continue
            args[param.name] = getattr(self, param.name)
        return args

    def is_running(self):
        """Check if the genetic algorithm should continue running."""
        stopping_condition_args = self.build_arg_dict(self.stopping_condition.is_running)
        return self.stopping_condition.is_running(**stopping_condition_args)

    def is_ready(self):
        """Check if the genetic algorithm is ready to run."""
        if (self.problem is None):
            raise ValueError("No problem has been set.")
        if (len(self.algorithm_steps) == 0):
            raise ValueError("No algorithm steps have been set.")
        if (self.stopping_condition is None):
            raise ValueError("No stopping condition has been set.")
        return True

    

In [156]:
# define genetic algorithm parameters in a dictionary
ga_parameters = {
  'problem' : De_Jong_Function_5(),
  'algorithm_steps' : [Selection(), Crossover(), Mutation()],
  'stopping_condition' : Max_Generations(),
  'population_size' : 100,
  
}

# create a genetic algorithm object
ga = Genetic_Algorithm(problem=De_Jong_Function_5())

In [157]:
ga.add_step(Selection())
